In [ ]:
class SEAICE_timeseries:   
    '''
    class SEAICE_timeseries(runname,resultpath,savepath,mesh,first_year,last_year,savefig=False,regional='N')
    
    Output: class Summer Sea-Ice Extent [million km2]
    Calculated and returned within required region:
    (N = "Arctic", S = "Southern")
    type can be area or extent.
    '''
    def __init__(self,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 savefig=False,regional='N',type = 'area', plotting=True, output=False):

        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.fyear = first_year
        self.lyear = last_year
        self.savefig = savefig
        self.regional = regional
        self.plotting = plotting
        self.ncpath = ncpath
        self.type = type

        from glob import glob as gg
        import matplotlib.pyplot as plt
        import numpy as np
        import pandas as pd
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from Py_f2recom_toolbox import plt_Taylor_norm

        print('## Observations only for the 1979-2019 period ##')
        years = np.arange(self.fyear, self.lyear+1,1)
        files = gg(self.ncpath, recursive = True)
        files = np.sort(files)

        ice = pf.get_data(resultpath,'a_ice', years, mesh, how=None, compute=False, silent=True)
 
        if regional == 'N':
            df = pd.read_csv(files[8], sep='\s*,\s*', index_col = 'year', parse_dates=True) #
            df = df.loc[((df.index.year>=self.fyear) & (df.index.year<=self.lyear))]
        elif regional == 'S':
            df = pd.read_csv(files[14], sep='\s*,\s*', index_col = 'year', parse_dates=True) #
            df = df.loc[((df.index.year>=self.fyear) & (df.index.year<=self.lyear))]
        
        if type == 'area':
            ice = pf.ice_area(ice, mesh, hemisphere=self.regional)
        elif type == 'extent':
            ice = pf.ice_ext(ice, mesh, hemisphere=self.regional)

        if regional == 'N':
            ice= ice.loc[ice['time'].dt.month==9]
        elif regional == 'S':
            ice= ice.loc[ice['time'].dt.month==3]
        
        fig = plt.figure(figsize=(8,5), constrained_layout=True)
        plt.plot(years, ice/1e12, label = 'model', lw =3)
        plt.plot(years, df[self.type], label = 'satellite', lw =3)
        if regional == 'N':
            plt.title('September Arctic Ocean sea-ice')
        elif regional == 'S':
            plt.title('March Southern Ocean sea-ice')
        plt.legend()
        plt.ylabel('sea-ice '+self.type+' [million km$^{-2}$]')
        plt.xlabel('time [year]')

        if self.savefig:
            plt.savefig('SEAICE.png', dpi = 300, bbox_inches='tight')
            plt.savefig('SEAICE.pdf', bbox_inches='tight') 